In [3]:
# app.py
# Interactive FRC robot cycle-time heatmap with Plotly Dash

import numpy as np
from heapq import heappush, heappop
from dataclasses import dataclass
from typing import List, Tuple, Optional

import dash
from dash import dcc, html, Output, Input, State, ctx
import plotly.graph_objects as go

# -----------------------------
# Grid model
# -----------------------------
@dataclass
class GridModel:
    w: int = 10
    h: int = 10
    robot: Tuple[int, int] = (5, 4)  # row, col
    blocked: List[Tuple[int, int]] = None

    def __post_init__(self):
        if self.blocked is None:
            self.blocked = [(3,3),(3,4),(3,5),(4,5),(5,5),(6,5),(6,4),(6,3),(2,7),(7,2)]

    def is_blocked(self, r: int, c: int) -> bool:
        return (r, c) in set(self.blocked)

    def toggle_blocked(self, r: int, c: int):
        if (r, c) == self.robot:
            return  # do not block the robot cell
        if (r, c) in self.blocked:
            self.blocked.remove((r, c))
        else:
            self.blocked.append((r, c))

    def set_robot(self, r: int, c: int):
        if (r, c) in self.blocked:
            # if user puts robot on blocked cell, un-block it first
            self.blocked.remove((r, c))
        self.robot = (r, c)

# -----------------------------
# Pathfinding (Dijkstra 4-neighbor)
# -----------------------------
def dijkstra_distances(w: int, h: int, start: Tuple[int, int], blocked: List[Tuple[int, int]]) -> np.ndarray:
    blocked_set = set(blocked)
    sr, sc = start
    dist = np.full((h, w), np.inf, dtype=float)
    dist[sr, sc] = 0.0
    pq = [(0.0, sr, sc)]
    while pq:
        d, r, c = heappop(pq)
        if d > dist[r, c]:
            continue
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < h and 0 <= nc < w and (nr, nc) not in blocked_set:
                nd = d + 1.0
                if nd < dist[nr, nc]:
                    dist[nr, nc] = nd
                    heappush(pq, (nd, nr, nc))
    return dist

# -----------------------------
# Figure builder
# -----------------------------
def build_figure(model: GridModel) -> go.Figure:
    dist = dijkstra_distances(model.w, model.h, model.robot, model.blocked)

    # matrix to display: NaN for blocked, 0 at robot for clarity
    viz = dist.astype(float).copy()
    for r, c in model.blocked:
        viz[r, c] = np.nan
    rr, cc = model.robot
    viz[rr, cc] = 0.0

    x_vals = list(range(model.w))
    y_vals = list(range(model.h))

    zmin = np.nanmin(viz)
    zmax = np.nanmax(viz)

    heat = go.Heatmap(
        z=viz,
        x=x_vals, y=y_vals,
        colorscale="Viridis",
        zmin=zmin, zmax=zmax,
        colorbar=dict(title="Steps"),
        hovertemplate="Row %{y}<br>Col %{x}<br>Steps %{z:.0f}<extra></extra>",
    )

    # black tiles for blocked
    mask = np.full_like(viz, np.nan, dtype=float)
    for r, c in model.blocked:
        mask[r, c] = 1.0
    blocked_layer = go.Heatmap(
        z=mask,
        x=x_vals, y=y_vals,
        colorscale=[[0, "black"], [1, "black"]],
        showscale=False, hoverinfo="skip",
        zmin=1, zmax=1, opacity=1.0,
    )

    robot = go.Scatter(
        x=[cc], y=[rr],
        mode="markers",
        marker=dict(symbol="star", size=16, line=dict(color="white", width=1.6)),
        name="Robot",
        hovertemplate="Robot<br>Row %{y}<br>Col %{x}<extra></extra>",
    )

    fig = go.Figure(data=[heat, blocked_layer, robot])
    fig.update_layout(
        title="FRC Robot Cycle Time Heatmap",
        xaxis=dict(title="Column (x)", tickmode="array", tickvals=x_vals, showgrid=True, gridcolor="lightgray"),
        yaxis=dict(title="Row (y)", tickmode="array", tickvals=y_vals, showgrid=True, gridcolor="lightgray", autorange="reversed"),
        width=760, height=760, margin=dict(l=60, r=20, t=60, b=60),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    return fig

# -----------------------------
# Dash app
# -----------------------------
app = dash.Dash(__name__)
app.title = "FRC Heatmap"

# Stores
# - model_state: the whole model (robot + blocked)
# - ui_mode: "robot" or "blocked"
default_model = GridModel()

app.layout = html.Div(
    style={"fontFamily": "Inter, system-ui, Segoe UI, Arial", "maxWidth": "1100px", "margin": "24px auto"},
    children=[
        html.H2("FRC Robot Cycle Time Heatmap"),
        html.P("Click a cell to place the robot or to toggle obstacles. Use the mode buttons to switch."),
        html.Div([
            html.Button("Robot mode", id="btn-robot", n_clicks=0, className="btn"),
            html.Button("Blocked mode", id="btn-blocked", n_clicks=0, className="btn", style={"marginLeft": "8px"}),
            html.Button("Clear obstacles", id="btn-clear", n_clicks=0, className="btn", style={"marginLeft": "16px"}),
            html.Button("Reset", id="btn-reset", n_clicks=0, className="btn", style={"marginLeft": "8px"}),
            html.Span(id="mode-label", style={"marginLeft": "16px", "fontWeight": "600"}),
        ], style={"marginBottom": "12px"}),

        dcc.Graph(id="heatmap", style={"border": "1px solid #ddd", "borderRadius": "10px"}),

        dcc.Store(id="model_state", data={
            "w": default_model.w,
            "h": default_model.h,
            "robot": list(default_model.robot),
            "blocked": [list(p) for p in default_model.blocked],
        }),
        dcc.Store(id="ui_mode", data="robot"),  # "robot" or "blocked"
    ]
)

# Update mode label when mode changes
@app.callback(
    Output("mode-label", "children"),
    Input("ui_mode", "data"),
)
def show_mode(mode):
    return f"Current mode: {mode.capitalize()}"

# Mode switching and clear/reset buttons
@app.callback(
    Output("ui_mode", "data", allow_duplicate=True),
    Output("model_state", "data", allow_duplicate=True),
    Input("btn-robot", "n_clicks"),
    Input("btn-blocked", "n_clicks"),
    Input("btn-clear", "n_clicks"),
    Input("btn-reset", "n_clicks"),
    State("ui_mode", "data"),
    State("model_state", "data"),
    prevent_initial_call=True,
)
def on_toolbar(robot_clicks, blocked_clicks, clear_clicks, reset_clicks, mode, data):
    trigger = ctx.triggered_id
    model = GridModel(
        w=data["w"], h=data["h"],
        robot=tuple(data["robot"]),
        blocked=[tuple(x) for x in data["blocked"]]
    )
    if trigger == "btn-robot":
        return "robot", data
    if trigger == "btn-blocked":
        return "blocked", data
    if trigger == "btn-clear":
        model.blocked = []
        return mode, {"w": model.w, "h": model.h, "robot": list(model.robot), "blocked": [list(p) for p in model.blocked]}
    if trigger == "btn-reset":
        model = GridModel()  # back to defaults
        return "robot", {"w": model.w, "h": model.h, "robot": list(model.robot), "blocked": [list(p) for p in model.blocked]}
    return dash.no_update, dash.no_update

# Handle clicks on the heatmap: place robot or toggle block
@app.callback(
    Output("model_state", "data"),
    Input("heatmap", "clickData"),
    State("ui_mode", "data"),
    State("model_state", "data"),
    prevent_initial_call=True,
)
def on_click(click_data, mode, data):
    if not click_data or "points" not in click_data or not click_data["points"]:
        return dash.no_update
    pt = click_data["points"][0]
    r = int(pt["y"])
    c = int(pt["x"])

    model = GridModel(
        w=data["w"], h=data["h"],
        robot=tuple(data["robot"]),
        blocked=[tuple(x) for x in data["blocked"]]
    )

    if mode == "robot":
        model.set_robot(r, c)
    else:
        model.toggle_blocked(r, c)

    return {"w": model.w, "h": model.h, "robot": list(model.robot), "blocked": [list(p) for p in model.blocked]}

# Redraw figure whenever the model changes
@app.callback(
    Output("heatmap", "figure"),
    Input("model_state", "data"),
)
def redraw(data):
    model = GridModel(
        w=data["w"], h=data["h"],
        robot=tuple(data["robot"]),
        blocked=[tuple(x) for x in data["blocked"]]
    )
    return build_figure(model)

if __name__ == "__main__":
    app.run_server(debug=True)


ObsoleteAttributeException: app.run_server has been replaced by app.run